# Import

In [1]:
# System
import os
import time
import json

# Data processing
import numpy as np
import math

# Plot
import matplotlib.pyplot as plt

# ML
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import StepLR
torch.set_num_threads(4)

# Config

In [2]:
# Config path
root = '/Volumes/Expansion/User_Backup/b08209033/111-2_IVT_analysis/'
folder = '2023_0321'
file = 'src/config.json'
config_path = os.path.join(os.path.join(root, folder), file)

# Import config
with open(config_path) as infile:
    config = json.load(infile)
    infile.close()

# Update config
config.update({"ML_batch_size": 8})
config.update({"ML_n_epochs": 10000})
config.update({"ML_learning_rate": 5e-4})
config.update({"ML_weight_decay": 0})
config.update({"ML_early_stop": 1000})
config.update({"ML_model_path": os.path.join(config["SubFolderPath"], "models")})
config.update({"ML_model_name": "model.pt"})

# Export config
with open(config_path, 'w') as outfile:
    json.dump(config, outfile, sort_keys=True)
    outfile.close()

## Config (ML)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# Model

In [4]:

class My_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(My_Model, self).__init__()
        self.GRU = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.GRU.weight_ih_l0.data.fill_(0)
        self.GRU.weight_hh_l0.data.fill_(0)
        self.input_length = input_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_size
        
    def forward(self, x, hidden_state):
        hidden_state[-1] = 0
        output, hidden_state = self.GRU(x, hidden_state)
        return output, hidden_state
        #return output, x[:,0,:].reshape(1,-1,self.hidden_dim)
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = torch.ones(self.num_layers, batch_size, self.hidden_dim).to(device)
        #hidden = weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

# Undetermined blocks

In [5]:
# Encapsulate to Dataloader
os.chdir(config["IVTPath"])
isDrop = True
train_loader = DataLoader(torch.load(config["ML_train_set_name"]), 
                          batch_size = config["ML_batch_size"],
                          drop_last = isDrop,
                          shuffle = False,
                          pin_memory = True)
valid_loader = DataLoader(torch.load(config["ML_valid_set_name"]), 
                          batch_size = config["ML_batch_size"], 
                          drop_last = isDrop,
                          shuffle = False,
                          pin_memory = True)
test_loader = DataLoader(torch.load(config["ML_test_set_name"]), 
                         batch_size = config["ML_batch_size"], 
                         drop_last = isDrop,
                         shuffle = False,
                         pin_memory = True)
with np.load(config["IVT_SVD_fname"]) as dataset:
    feature_num = int(dataset['feature_threshold'][0][0])


In [6]:
def trainer(train_loader, valid_loader, model, config, device):
    
    # Pre train stage
        # model_dict
    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.
        # train parameter
    n_epochs, best_loss, step, early_stop_count = config['ML_n_epochs'], math.inf, 0, 0
    
    # Loss function
    criterion = nn.L1Loss(reduction='mean').to(device)
    # Optimizer
    
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr = config['ML_learning_rate'], 
                                 weight_decay = config['ML_weight_decay'])
    
    scheduler_Cycle = CosineAnnealingWarmRestarts(optimizer, T_0 = 50, T_mult = 5, eta_min = config['ML_weight_decay']/1e4)
    scheduler_Decay = StepLR(optimizer, step_size = 50, gamma = 0.3)
        # Training & Validating
    mean_train_loss_record = []
    mean_valid_loss_record = []
    for epoch in range(n_epochs):
        # Training stage
        # Init
        model.train()
        loss_record = []
        hidden = model.init_hidden(config['ML_batch_size'])
        for batch, (x, y, t) in enumerate(train_loader):
            # Reset gradient
            optimizer.zero_grad()
            #hidden = model.init_hidden(config['ML_batch_size'])
            # cuda if possible
            x, y = x.to(device), y.to(device)
            # Modify batch size in hidden state
            #hidden = model.init_hidden(config['ML_batch_size'])
            pred, hidden = model(x, hidden)
            # Calculate loss
            loss = criterion(pred[:,-1,:], y[:,-1,:])
            # Backward propagation
            loss.backward()
            
            
            # Update model parameter
            optimizer.step()
            
            
            step += 1
            # Detach unused graph
            hidden.detach_()
            loss_record.append(loss.detach().item())
        #scheduler_Cycle.step()
        scheduler_Decay.step()
        # Train loss
        mean_train_loss = sum(loss_record)/len(loss_record)
        mean_train_loss_record.append(mean_train_loss)
        
        # Validating stage
        # Init
        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y, t in valid_loader:
            # cuda if possible
            x, y = x.to(device), y.to(device)
            # Modify batch size in hidden state
            #hidden = model.init_hidden(config['ML_batch_size'])
            # Skip gradient update and backward propagation
            with torch.no_grad():
                # Forward propagation
                pred, hidden = model(x, hidden)
                # Calculate loss
                loss = criterion(pred[:,-1,:], y[:,-1,:])
            
            # Detach loss
            loss_record.append(loss.item())
            
        # Valid loss
        mean_valid_loss = sum(loss_record)/len(loss_record)
        mean_valid_loss_record.append(mean_valid_loss)
        
        # Show progress
        if (epoch%10==0):
            print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.7f}, Valid loss: {mean_valid_loss:.7f}')
        
        # Save model parameter
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            os.chdir(config['ML_model_path'])
            torch.save(model, config['ML_model_name']) # Save your best model
            # print('Saving model with loss {:.5f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1
            
        # Early stop
        if early_stop_count >= config['ML_early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return hidden, mean_train_loss_record, mean_valid_loss_record
    return hidden, mean_train_loss_record, mean_valid_loss_record


In [7]:
os.chdir(config["FolderPath"])
model = My_Model(input_size = feature_num,
                 hidden_size = feature_num,
                 num_layers = 1).to(device) # put your model and data on the same computation device.

hidden, train_loss, valid_loss = trainer(train_loader, valid_loader, model, config, device)

Epoch [1/10000]: Train loss: 0.1226401, Valid loss: 0.1080412
Epoch [11/10000]: Train loss: 0.0785357, Valid loss: 0.0840111
Epoch [21/10000]: Train loss: 0.0777352, Valid loss: 0.0836152
Epoch [31/10000]: Train loss: 0.0774326, Valid loss: 0.0836049
Epoch [41/10000]: Train loss: 0.0772116, Valid loss: 0.0836159
Epoch [51/10000]: Train loss: 0.0766482, Valid loss: 0.0834582
Epoch [61/10000]: Train loss: 0.0764804, Valid loss: 0.0834265
Epoch [71/10000]: Train loss: 0.0764338, Valid loss: 0.0834400


KeyboardInterrupt: 

In [ ]:
for name,param in model.named_parameters():
    print(name)
    print(param.size())

In [ ]:

model.eval() # Set your model to evaluation mode.
preds = []
targets = []
times = []
hidden = model.init_hidden(config['ML_batch_size'])
for i, (x,y,t) in enumerate(test_loader):
    x = x.to(device)
    y = y.to(device)
    t = t.to(device)
    with torch.no_grad():
        pred, hidden = model(x, hidden)
        preds.append(pred.detach().cpu()[:,-1,:])
        targets.append(y.detach().cpu()[:,-1,:])
        times.append(t.detach().cpu()[:,-1,:])
preds = torch.cat(preds, dim=0).numpy()
targets = torch.cat(targets, dim=0).numpy()
times = torch.cat(times, dim=0).numpy()


In [ ]:
"""
os.chdir(config["ImgPath"])
t_interval = times[:,-1] - times[0,-1]
for idx in range(feature_num):
    plt.figure(figsize=(12,8), dpi = 200)
    plt.plot(t_interval,preds[:,idx], label = "predict", zorder = 3)
    plt.plot(t_interval,targets[:,idx], label = "target", zorder = 2)
    plt.legend(loc = 1, prop={'size': 20})
    plt.title(f"Time structure of filtered SVD Spatial mode:{idx}")
    plt.savefig(f"Time series, mode:{idx}")
    plt.close()
"""

# Unused blocks